In [1]:
import pandas as pd
import operator
import re
from collections import defaultdict

doc = pd.read_csv('sample_dataset.csv', sep=';')

doc.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-14 10:26,https://twitter.com/dekoran1/status/9082454469...,NaN,post,NaN,11369.0,dekoran1,11369,131.1,NaN,NaN,@BLIKOPENER333,twitter,NaN
1,ziek src:twitter,2017-09-14 10:25,https://twitter.com/BLIKOPENER333/status/90824...,-,comment,2.0,17592.0,BLIKOPENER333,17592,9.1,NaN,NaN,RT @tikkierechts: Ziek of niet dit is echt een...,twitter,NaN
2,ziek src:twitter,2017-09-14 10:24,https://twitter.com/Jesske3003/status/90824496...,-,comment,19.0,NaN,Jesske3003,856,4.5,NaN,NaN,@tinamieneke @DockxGunter Ja en elkaar wat zie...,twitter,NaN
3,ziek src:twitter,2017-09-14 10:23,https://twitter.com/tinamieneke/status/9082446...,-,comment,19.0,NaN,tinamieneke,1928,25.7,NaN,NaN,"@DockxGunter @Jesske3003 Ik vind dat niet oké,...",twitter,NaN
4,ziek src:twitter,2017-09-14 10:23,https://twitter.com/BeQuanFRIvoedin/status/908...,-,post,NaN,38.0,BeQuanFRIvoedin,38,0.6,NaN,NaN,Hippe huiskip maakte nog nooit zoveel Amerikan...,twitter,NaN


Filter out retweets since they are not relevant to our research

In [2]:
doc[~doc['bericht tekst'].str.contains('rt', case=False)]

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-14 10:26,https://twitter.com/dekoran1/status/9082454469...,NaN,post,NaN,11369.0,dekoran1,11369,131.1,NaN,NaN,@BLIKOPENER333,twitter,NaN
3,ziek src:twitter,2017-09-14 10:23,https://twitter.com/tinamieneke/status/9082446...,-,comment,19.0,NaN,tinamieneke,1928,25.7,NaN,NaN,"@DockxGunter @Jesske3003 Ik vind dat niet oké,...",twitter,NaN
4,ziek src:twitter,2017-09-14 10:23,https://twitter.com/BeQuanFRIvoedin/status/908...,-,post,NaN,38.0,BeQuanFRIvoedin,38,0.6,NaN,NaN,Hippe huiskip maakte nog nooit zoveel Amerikan...,twitter,NaN
5,ziek src:twitter,2017-09-14 10:19,https://twitter.com/BillyBibixx/status/9082436...,-,post,NaN,775.0,BillyBibixx,775,5.3,NaN,NaN,"Ik ga naar huis, dit is echt kut",twitter,NaN
6,ziek src:twitter,2017-09-14 10:18,https://twitter.com/flemmishfan/status/9082436...,-,comment,8.0,NaN,flemmishfan,24,1.3,NaN,NaN,@demorgen Mijn idee ook kinderen mogen zich ni...,twitter,NaN
7,ziek src:twitter,2017-09-14 10:17,https://twitter.com/DockxGunter/status/9082433...,+,comment,19.0,NaN,DockxGunter,81,6.2,NaN,NaN,@tinamieneke @Jesske3003 Ik ben redelijk ziek ...,twitter,NaN
12,ziek src:twitter,2017-09-14 10:10,https://twitter.com/c3626/status/9082416155878...,-,comment,4.0,NaN,c3626,195,2.0,NaN,NaN,@ChrisVonMokum @pino_chetsnek Wat maakt dit ub...,twitter,NaN
15,ziek src:twitter,2017-09-14 10:08,https://twitter.com/Smarti_s/status/9082411153...,-,post,NaN,313.0,Smarti_s,313,1.4,NaN,NaN,Graftakken kop vandaag omdat ik zonder make-up...,twitter,NaN
16,ziek src:twitter,2017-09-14 10:08,https://twitter.com/tinamieneke/status/9082409...,+,comment,NaN,NaN,tinamieneke,1928,25.7,NaN,NaN,@DearRipley Omdat zij anders nooit ziek is heb...,twitter,NaN
18,ziek src:twitter,2017-09-14 10:06,https://twitter.com/theycallmetara/status/9082...,-,post,NaN,4300.0,theycallmetara,4300,2.0,NaN,NaN,die nuon reclame heeft me nu al 3x laten janke...,twitter,NaN


Lower all text so for consistency

In [3]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [4]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+)')

def clean_text(text: str):
    text = re_clean.sub(' ', text)
    words = re_words.findall(text)
    return words

In [5]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

words = defaultdict(int)

for tweet_words in all_tweets['bericht woorden']:
    for word in tweet_words:
        words[word] += 1
            
words = sorted(words.items(), key=operator.itemgetter(1))[::-1]
blacklisted_words = dict(words[:100]).keys()
blacklisted_words

dict_keys(['RT', 'de', 'in', 'is', 'van', 'en', 'een', 'op', 'het', 'voor', 'met', 'the', 'je', 'to', 'niet', 'of', 'te', 'ik', 's', 'a', 'dat', 'zijn', 'MCLISSE', 'OnTGSBlogCon', 'die', 'aan', 'and', 'Ik', 'I', 'bij', 'you', 'er', 'for', 'De', 'via', 'ook', 'over', '2', 'maar', 'on', 'om', 'dan', '1', 'naar', 'als', 'nog', 'it', '3', 'this', 'me', 'wat', 'door', 'uit', 'we', 'with', 'nu', 'ze', 'En', 'at', 'wel', 'Het', 'zo', 'The', 'was', '0', 'meer', 'dit', 'na', 'mijn', 'your', 't', 'kan', 'al', 'weer', 'We', 'gaat', 'deze', '4', 'heeft', 'by', 'jaar', 'geen', 'Nederland', 'that', 'wil', '12', 'u', 'Een', 'haar', 'heb', 'ben', '2017', 'Wat', 'be', 'my', 'Amsterdam', 'Dat', '10', 'hebben', 'nieuwe'])

In [6]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)
words = defaultdict(int)

for tweet_words in doc['bericht woorden']:
    for word in tweet_words:
        if word not in blacklisted_words:
            words[word] += 1
                
words = sorted(words.items(), key=operator.itemgetter(1))[::-1]
words[:50]
# doc[doc['bericht tekst'].str.contains('eigenrisico')].filter(items=['bericht tekst', 'index'])

[('ziek', 9588),
 ('rt', 4636),
 ('hoe', 1047),
 ('eigenrisico', 886),
 ('eigen', 732),
 ('worden', 706),
 ('boete', 613),
 ('moet', 597),
 ('risico', 593),
 ('echt', 577),
 ('thuis', 558),
 ('mensen', 491),
 ('ga', 467),
 ('wordt', 450),
 ('kunde', 448),
 ('laten', 418),
 ('zorgfonds', 412),
 ('dus', 410),
 ('maken', 397),
 ('agenten', 394),
 ('kind', 342),
 ('tis', 336),
 ('chronisch', 335),
 ('winter', 334),
 ('overleven', 330),
 ('fcking', 328),
 ('zomer', 327),
 ('tfqq', 326),
 ('tafel', 324),
 ('schandalige', 322),
 ('veel', 312),
 ('bent', 310),
 ('zelf', 308),
 ('gebeuren', 298),
 ('ouders', 292),
 ('zwaarder', 279),
 ('daar', 274),
 ('stopdeboeteopziekzijn', 273),
 ('beboeten', 273),
 ('politie', 262),
 ('want', 259),
 ('toch', 249),
 ('zich', 249),
 ('verder', 248),
 ('word', 243),
 ('keuze', 237),
 ('zitten', 231),
 ('mee', 224),
 ('omhoog', 221),
 ('hij', 216)]

In [7]:
word_frame = pd.DataFrame(words)
word_frame.to_csv('words.csv')
word_frame

,0,1
0,ziek,9588
1,rt,4636
2,hoe,1047
3,eigenrisico,886
4,eigen,732
5,worden,706
6,boete,613
7,moet,597
8,risico,593
9,echt,577
